In [2]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=2811633 sha256=396a56f5d4d2ea6d6b8efbc974e9d1c3092dbd673ae48110104167fb5aebd6ad
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval #parse the stringified features into their corresponding python objects
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
from surprise.prediction_algorithms.knns import KNNBasic
from surprise.model_selection.validation import cross_validate
from surprise.model_selection import GridSearchCV, RandomizedSearchCV

sns.set_style()

import warnings; warnings.simplefilter('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
md = pd. read_csv('drive/MyDrive/the_movies_dataset/movies_metadata.csv')
md.head().transpose()

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col..."
budget,30000000,65000000,0,16000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 35, 'name': 'Comedy'}]"
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...


In [6]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval) \
                           .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

md['production_companies']= md['production_companies'].fillna('[]') \
                                                      .apply(literal_eval) \
                                                      .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['production_countries'] = md['production_countries'].fillna('[]') \
                                                       .apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['spoken_languages'] = md['spoken_languages'].fillna('[]') \
                                               .apply(literal_eval) \
                                               .apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [8]:
md.head(10).transpose()

,0,1,2,3,4,5,6,7,8,9
adult,False,False,False,False,False,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,NaN,NaN,NaN,"{'id': 645, 'name': 'James Bond Collection', '..."
budget,30000000,65000000,0,16000000,0,60000000,58000000,0,35000000,58000000
genres,"[Animation, Comedy, Family]","[Adventure, Fantasy, Family]","[Romance, Comedy]","[Comedy, Drama, Romance]",[Comedy],"[Action, Crime, Drama, Thriller]","[Comedy, Romance]","[Action, Adventure, Drama, Family]","[Action, Adventure, Thriller]","[Adventure, Action, Thriller]"
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.mgm.com/view/movie/757/Goldeneye/
id,862,8844,15602,31357,11862,949,11860,45325,9091,710
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041,tt0113277,tt0114319,tt0112302,tt0114576,tt0113189
original_language,en,en,en,en,en,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II,Heat,Sabrina,Tom and Huck,Sudden Death,GoldenEye
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...,"Obsessive master thief, Neil McCauley leads a ...",An ugly duckling having undergone a remarkable...,"A mischievous young boy, Tom Sawyer, witnesses...",International action superstar Jean Claude Van...,James Bond must unmask the mysterious head of ...


#Top-k Recommender

**Рекомендации на основе топ-k популярных фильмов в выбранном жанре.** \
Также можно советовать топы фильмов по другим признакам, таким как страна, год выпуска, режиссер и тд. Кроме того, пользователю можно предлагать топы фильмов конкретно для его категории, например в зависимости от пола и возраста или от его местоположения.

Метрика WR \\
$WR\ (Weighted\ Rating) = \frac{v}{v+m}R + \frac{m}{v+m}C$,
где
*   v - количество оценок фильма (vote_count);
*   m - минимальное количество голосов, необходимое для внесения в таблицу;
*   R - средний рейтинг фильма (vote_average);
*   C - средняя оценка для всей таблицы.

In [ ]:
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()
C

5.244896612406511

In [7]:
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
m = vote_counts.quantile(0.95)
m

434.0

In [ ]:
qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & \
               (md['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [ ]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [ ]:
qualified.head(15)

,title,year,vote_count,vote_average,popularity,genres,wr
15480,Inception,2010,14075,8,29.108149,"[Action, Thriller, Science Fiction, Mystery, A...",7.917588
12481,The Dark Knight,2008,12269,8,123.167259,"[Drama, Action, Crime, Thriller]",7.905871
22879,Interstellar,2014,11187,8,32.213481,"[Adventure, Drama, Science Fiction]",7.897107
2843,Fight Club,1999,9678,8,63.869599,[Drama],7.881753
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,"[Adventure, Fantasy, Action]",7.871787
292,Pulp Fiction,1994,8670,8,140.950236,"[Thriller, Crime]",7.868660
314,The Shawshank Redemption,1994,8358,8,51.645403,"[Drama, Crime]",7.864000
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,"[Adventure, Fantasy, Action]",7.861927
351,Forrest Gump,1994,8147,8,48.307194,"[Comedy, Drama, Romance]",7.860656
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,"[Adventure, Fantasy, Action]",7.851924


In [ ]:
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)

In [ ]:
def build_chart(genre, percentile=0.9):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)

    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & \
                   (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')

    qualified['wr'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)

    return qualified

In [ ]:
build_chart('Fantasy').head(15)

,title,year,vote_count,vote_average,popularity,wr
4863,The Lord of the Rings: The Fellowship of the Ring,2001,8892,8,32.070725,7.781409
7000,The Lord of the Rings: The Return of the King,2003,8226,8,29.324358,7.765292
5814,The Lord of the Rings: The Two Towers,2002,7641,8,29.423537,7.749038
3030,The Green Mile,1999,4166,8,19.96678,7.573666
5481,Spirited Away,2001,3968,8,41.048867,7.555987
9698,Howl's Moving Castle,2004,2049,8,16.136048,7.257625
2884,Princess Mononoke,1997,2041,8,17.166725,7.255540
5833,My Neighbor Totoro,1988,1730,8,13.507299,7.164272
14551,Avatar,2009,12114,7,185.070892,6.898360
926,It's a Wonderful Life,1946,1103,8,15.031588,6.889898


#Content Based Recommender

In [9]:
links_small = pd.read_csv('drive/MyDrive/the_movies_dataset/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [10]:
md = md.drop([19730, 29503, 35587]) #id is None

In [11]:
md['id'] = md['id'].astype(int)

In [12]:
smd = md[md['id'].isin(links_small)]
smd.shape

(9099, 25)

##Рекомендации по описанию. Movie Description Based Recommender.

In [13]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

In [14]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [15]:
tfidf_matrix.shape

(9099, 268124)

Используем косинусное сходство *(cosine similarity)* для вычисления сходства между двумя фильмами.
$${\displaystyle {\text{cosine similarity}}:={\mathbf {x} \cdot \mathbf {y^T}  \over \|\mathbf {x} \|\|\mathbf {y} \|}}$$

In [16]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [17]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [18]:
def get_recommendations(title, num=30):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num+1]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [19]:
pd.DataFrame(get_recommendations('Pulp Fiction')).head(10)

,title
7977,Holding Trevor
6974,The Wackness
2935,Prizzi's Honor
6734,You Kill Me
240,Little Odessa
562,Moll Flanders
6837,Wristcutters: A Love Story
5416,La Jetée
7433,The First Day of the Rest of Your Life
8739,Comet


##Рекомендации на основе данных о фильме. Metadata Based Recommender.

In [20]:
credits = pd.read_csv('drive/MyDrive/the_movies_dataset/credits.csv')
keywords = pd.read_csv('drive/MyDrive/the_movies_dataset/keywords.csv')

In [21]:
credits.head(10)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [22]:
keywords.head(10)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
5,949,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
6,11860,"[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam..."
7,45325,[]
8,9091,"[{'id': 949, 'name': 'terrorist'}, {'id': 1562..."
9,710,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam..."


In [23]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [24]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')
smd = md[md['id'].isin(links_small)]
smd.shape

(9219, 28)

In [25]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [26]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [27]:
smd['director'] = smd['crew'].apply(get_director)

In [28]:
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:5] if len(x) >=5 else x)

In [29]:
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [30]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [31]:
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))

In [32]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'

In [33]:
s = s.value_counts()
s[:10]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
violence                264
love                    222
musical                 219
sex                     219
suspense                212
Name: keyword, dtype: int64

In [34]:
s = s[s > 1]

In [35]:
stemmer = SnowballStemmer('english')

In [36]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [37]:
smd['keywords'] = smd['keywords'].apply(filter_keywords) \
                                 .apply(lambda x: [stemmer.stem(i) for i in x]) \
                                 .apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [38]:
smd.head().transpose()

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col..."
budget,30000000,65000000,0,16000000,0
genres,"[Animation, Comedy, Family]","[Adventure, Fantasy, Family]","[Romance, Comedy]","[Comedy, Drama, Romance]",[Comedy]
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...


In [ ]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x)) + smd['director']
pd.DataFrame(smd['soup']).head()

,soup
0,jealousi toy boy friendship friend rivalri boy...
1,boardgam disappear basedonchildren'sbook newho...
2,fish bestfriend duringcreditssting waltermatth...
3,basedonnovel interracialrelationship singlemot...
4,babi midlifecrisi confid age daughter motherda...


In [ ]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [ ]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [ ]:
pd.DataFrame(get_recommendations('Pulp Fiction')).head(10)

,title
7831,The Sunset Limited
616,The Great White Hype
6939,Cleaner
8558,Oldboy
4595,Basic
4306,The 51st State
3214,Unbreakable
7233,Afro Samurai: Resurrection
4764,S.W.A.T.
440,Menace II Society


Так же можно попробовать различные веса для режиссеров, актеров, жанров, etc., ограничивать количество ключевых слов, которые можно использовать в супе, взвешивать жанры на основе их частоты, показывать только фильмы с одинаковыми языками и т.д.

In [ ]:
def improved_recommendations(title, num=25, percentile=0.6):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num+1]
    movie_indices = [i[0] for i in sim_scores]

    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False)
    return qualified

In [ ]:
improved_recommendations('Pulp Fiction').head(10)

,title,vote_count,vote_average,year,wr
3214,Unbreakable,1994,6,2000,5.865027
440,Menace II Society,179,7,1993,5.757398
4595,Basic,290,6,2003,5.547355
6705,Black Snake Moan,247,6,2006,5.518774
4306,The 51st State,173,5,2001,5.175099
6939,Cleaner,201,5,2007,5.167378
3012,Shaft,316,5,2000,5.141714
8558,Oldboy,632,5,2013,5.099705
4764,S.W.A.T.,780,5,2003,5.087550
7104,The Spirit,323,4,2008,4.713719


#Collaborative Filtering

##Model Based
Используем библиотеку Surprise, используемую специально для работы с явными (explicit) оценками, смотри [документацию](https://surprise.readthedocs.io/en/stable/). При явном оценивании пользователь сам показывает, насколько ему интересен тот или иной объект.

###SVD

In [ ]:
reader = Reader()

In [ ]:
ratings = pd.read_csv('drive/MyDrive/the_movies_dataset/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True);

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9050  0.8890  0.8821  0.8980  0.8950  0.8992  0.8990  0.8888  0.8863  0.8915  0.8934  0.0067  
MAE (testset)     0.6941  0.6856  0.6788  0.6908  0.6893  0.6950  0.6893  0.6858  0.6792  0.6863  0.6874  0.0052  
Fit time          1.44    1.67    1.56    1.42    1.45    1.35    1.37    1.40    1.61    1.64    1.49    0.11    
Test time         0.05    0.33    0.05    0.16    0.05    0.06    0.06    0.05    0.07    0.05    0.09    0.09    


In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
ratings[ratings['userId'] == 1].head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
svd.predict(1, 31, 2.5)

Prediction(uid=1, iid=31, r_ui=2.5, est=2.484137999980176, details={'was_impossible': False})

Hyper-parameter tuning to find optimal value of *n_factors, reg_all and n_epochs*

In [ ]:
param_distributions  = {'n_factors': list(range(50,160,10)),'reg_all': np.arange(0.02,0.2,0.02),'n_epochs' : list(range(1,51))}
rs = RandomizedSearchCV(SVD, param_distributions, measures=['rmse'], return_train_measures = True, cv = 5, n_iter = 20)
rs.fit(data)
rs.best_params['rmse']

{'n_factors': 110, 'reg_all': 0.1, 'n_epochs': 48}

In [ ]:
#rs.best_params['rmse'] = {'n_factors': 110, 'reg_all': 0.1, 'n_epochs': 48}
tuned_svd_model = SVD(n_factors=110, reg_all=0.1, n_epochs=48, random_state=42,verbose=False)
cross_validate(tuned_svd_model, data, measures=['RMSE', 'MAE'], cv=10, verbose=True);

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.8711  0.8791  0.8872  0.8796  0.8633  0.8551  0.8825  0.8806  0.8749  0.8766  0.8750  0.0091  
MAE (testset)     0.6698  0.6752  0.6821  0.6771  0.6644  0.6580  0.6780  0.6765  0.6772  0.6748  0.6733  0.0069  
Fit time          3.66    3.56    3.46    3.34    3.89    3.30    3.33    3.78    3.31    3.22    3.49    0.22    
Test time         0.09    0.05    0.07    0.06    0.05    0.05    0.05    0.11    0.05    0.05    0.06    0.02    


In [ ]:
tuned_svd_model.fit(trainset)
tuned_svd_model.predict(1, 31, 2.5)

Prediction(uid=1, iid=31, r_ui=2.5, est=2.2287606066320897, details={'was_impossible': False})

##Memory Based
Также существуют Memory Based подходы, которые при рекомендациях опираются на user-user или item-item сходство (например, с помощью k-NN)

###User Based

In [ ]:
sim_options = {'name': 'cosine',
               'user_based': True}

sim_user = KNNBasic(sim_options=sim_options, verbose=False, random_state=42)

cross_validate(sim_user, data, measures=['RMSE', 'MAE'], cv=10, verbose=True);

Evaluating RMSE, MAE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9929  0.9867  0.9711  0.9863  0.9961  0.9914  0.9904  0.9931  1.0047  0.9855  0.9898  0.0082  
MAE (testset)     0.7667  0.7611  0.7521  0.7634  0.7694  0.7636  0.7635  0.7653  0.7767  0.7618  0.7644  0.0059  
Fit time          0.16    0.21    0.22    0.28    0.17    0.15    0.15    0.21    1.17    0.17    0.29    0.30    
Test time         0.73    0.73    0.70    0.93    0.69    0.63    0.68    1.04    1.10    1.07    0.83    0.17    


In [ ]:
sim_user.fit(trainset)
sim_user.predict(1, 31)

Prediction(uid=1, iid=31, r_ui=None, est=3.1834796860227086, details={'actual_k': 40, 'was_impossible': False})

###Item Based

In [ ]:
sim_options = {'name': 'cosine',
               'user_based': False}
sim_item = KNNBasic(sim_options=sim_options, verbose=False, random_state=42)

cross_validate(sim_item, data, measures=['RMSE', 'MAE'], cv=10, verbose=True);

Evaluating RMSE, MAE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9963  0.9855  0.9918  0.9911  0.9976  0.9899  1.0104  0.9965  1.0015  0.9906  0.9951  0.0067  
MAE (testset)     0.7758  0.7726  0.7752  0.7691  0.7720  0.7721  0.7806  0.7749  0.7790  0.7726  0.7744  0.0033  
Fit time          5.01    4.90    5.09    4.96    5.15    5.27    5.39    4.82    5.07    5.11    5.08    0.16    
Test time         3.97    3.39    3.15    3.97    2.91    3.51    3.33    3.67    3.15    3.20    3.42    0.34    


In [ ]:
sim_item.fit(trainset)
sim_item.predict(1, 31)

Prediction(uid=1, iid=31, r_ui=None, est=2.547471538910294, details={'actual_k': 20, 'was_impossible': False})

Hyper-parameter tuning using GridSearch

In [ ]:
param_grid = {'k': list(range(10,45,5)),
             'min_k' : list(range(5,11))}
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], return_train_measures = True, cv = 5)
gs.fit(data)
gs.best_params['rmse']

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

{'k': 15, 'min_k': 5}

In [ ]:
#gs.best_params['rmse'] == {'k': 15, 'min_k': 5}
tuned_sim_options = {'name': 'cosine',
               'user_based': False,
               'k': 15, 'min_k': 5}
tuned_knn_model = KNNBasic(sim_options=tuned_sim_options, verbose=False, random_state=42)

cross_validate(tuned_knn_model, data, measures=['RMSE', 'MAE'], cv=10, verbose=True);

Evaluating RMSE, MAE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.9964  0.9882  0.9970  1.0062  0.9985  0.9882  0.9912  0.9943  0.9946  0.9950  0.9950  0.0050  
MAE (testset)     0.7779  0.7711  0.7763  0.7772  0.7765  0.7708  0.7711  0.7747  0.7754  0.7768  0.7748  0.0026  
Fit time          4.95    7.55    5.63    4.20    4.65    4.71    4.26    4.68    4.97    4.31    4.99    0.94    
Test time         3.13    2.96    2.72    3.24    2.98    3.41    3.01    3.42    3.22    3.06    3.12    0.20    


In [ ]:
tuned_knn_model.fit(trainset)
tuned_knn_model.predict(1, 31)

Prediction(uid=1, iid=31, r_ui=None, est=2.547471538910294, details={'actual_k': 20, 'was_impossible': False})

#Hybrid Recommender

На вход алгоритм будет получать id юзера и название фильма, после чего будет выдан список фильмов, похожих на данный, отсортированных в порядке предсказанного рейтинга на основе коллаборативной фильтрации.

In [ ]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [ ]:
id_map = pd.read_csv('drive/MyDrive/the_movies_dataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
id_map.head()

,movieId,id
title,,
Toy Story,1,862.0
Jumanji,2,8844.0
Grumpier Old Men,3,15602.0
Waiting to Exhale,4,31357.0
Father of the Bride Part II,5,11862.0


In [ ]:
indices_map = id_map.set_index('id')
indices_map.head()

,movieId
id,
862.0,1
8844.0,2
15602.0,3
31357.0,4
11862.0,5


In [ ]:
def hybrid(userId, title, num=25):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']

    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num+1]
    movie_indices = [i[0] for i in sim_scores]

    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: tuned_knn_model.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies

In [ ]:
hybrid(1, 'Pulp Fiction').head(10)

,title,vote_count,vote_average,year,id,est
8558,Oldboy,632.0,5.9,2013,87516,3.543608
7104,The Spirit,323.0,4.7,2008,8285,3.543608
2745,School Daze,27.0,6.1,1988,36739,3.000000
4245,Betsy's Wedding,11.0,5.0,1990,18913,2.732130
3214,Unbreakable,1994.0,6.9,2000,9741,2.557139
3062,White Sands,30.0,5.5,1992,19380,2.555445
440,Menace II Society,179.0,7.2,1993,9516,2.547399
3012,Shaft,316.0,5.5,2000,479,2.546645
405,Fresh,56.0,7.6,1994,13815,2.544329
616,The Great White Hype,29.0,5.1,1996,20759,2.532066


In [ ]:
hybrid(42, 'Pulp Fiction').head(10)

,title,vote_count,vote_average,year,id,est
2745,School Daze,27.0,6.1,1988,36739,4.600000
7104,The Spirit,323.0,4.7,2008,8285,4.309524
7148,Soul Men,47.0,6.3,2008,14655,4.250000
231,Kiss of Death,83.0,5.7,1995,6071,4.241942
8964,Kite,98.0,4.2,2014,192712,4.200000
7868,African Cats,74.0,7.1,2011,57586,4.200000
4306,The 51st State,173.0,5.9,2001,1613,4.162500
4245,Betsy's Wedding,11.0,5.0,1990,18913,4.155272
6705,Black Snake Moan,247.0,6.6,2006,7874,4.150000
9038,Afro Samurai,63.0,7.3,2007,62931,4.150000


In [ ]:
!pip freeze > requirements.txt